https://docs.ragas.io/en/stable/getstarted/rag_testset_generation/#load-documents

In [2]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, AnswerRelevancy, FactualCorrectness
from langchain_community.chat_models import ChatOllama 
from ragas.llms import LangchainLLMWrapper
from langchain_community.chat_models import ChatOllama
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd


/Users/fedecl/Library/CloudStorage/Dropbox/llm-ceres/env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:

llm = ChatOllama(model="llama3")  
llm_wrapper = LangchainLLMWrapper(llm)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L12-v2")

/var/folders/m4/ql4hzm856d50x_6kl4l3g3mr0000gn/T/ipykernel_58365/2476231286.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")
/var/folders/m4/ql4hzm856d50x_6kl4l3g3mr0000gn/T/ipykernel_58365/2476231286.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers

# Evaluación usando preguntas sin respuesta esperada

In [64]:
df = pd.read_csv("interacciones.csv")
df.head()

,user_input,retrieved_contexts,response
0,¿Quién fue Joaquín Sorolla?,"Joaquín Sorolla y Bastida (Valencia, 27 de feb...",Joaquín Sorolla y Bastida fue un pintor españo...
1,¿Qué importancia tiene la luz en la pintura de...,. La amplitud de esta sala bañada por la luz n...,No he encontrado información específica sobre ...
2,¿Cuál es la historia del Museo Sorolla?,# Datos identificativos del Museo: El Museo So...,La historia del Museo Sorolla comienza con el ...
3,¿Dónde se encuentra la casa de Sorolla?,# El edificio del museo: la casa Sorolla. Desd...,La casa de Sorolla se encuentra en el Paseo de...
4,Enumera 5 fechas de la vida de Joaquín Sorolla,# Fechas importantes de la vida de Joaquín Sor...,"A continuación, te proporciono 5 fechas import..."


In [69]:
df.iloc[12]

user_input            ¿Qué técnicas de conservación se aplican a las...
retrieved_contexts    # Quejas y sugerencias\nLos usuarios podrán pr...
response              No he encontrado información sobre las técnica...
Name: 12, dtype: object

In [71]:
for i in df.iloc[9]:
    print(i)

¿Cuál es el contexto histórico del impresionismo?
En agosto de 1900, estando en Valencia, su amigo el escultor Ricardo Causarás Casaña fue a visitar a Sorolla para pedirle que posara de modelo vivo para esculpir una estatua de terracota y yeso, algo mayor del natural, que Causarás tenía pensado exponer en la Exposición Nacional de Bellas Artes de Madrid del año 1901

.​ Al acabar su formación comenzó a enviar sus obras a concursos provinciales y exposiciones nacionales de bellas artes, como la de Madrid en mayo de 1881, donde presentó tres marinas valencianas que pasaron inadvertidas, pues no encajaban con la pintura oficial, de temática histórica y dramática.

#El encargo de la Hispanic Society: En noviembre de ese mismo año firmó un encargo para la Sociedad Hispánica de América, por el que realizaría catorce murales que decorarían las salas de la institución, y dedicados a las Regiones de España. Con esta obra realizada entre 1913 y 1919, de tres metros y medio de alto por setenta me

In [6]:
dataset = []
for i, row in df.iterrows():
    entry = {
        "user_input": row['user_input'],
        "retrieved_contexts": row['retrieved_contexts'] if isinstance(row['retrieved_contexts'], list) else [row['retrieved_contexts']],
        "response": row['response']
    }
    dataset.append(entry)

my_eval_dataset = EvaluationDataset.from_list(dataset)

In [33]:
from ragas.metrics import AnswerRelevancy, ContextRelevance, Faithfulness, ResponseGroundedness


resultados = evaluate(
    dataset=my_eval_dataset,
    metrics=[
        AnswerRelevancy(), ContextRelevance(), Faithfulness(), ResponseGroundedness()
        ],
    llm=llm_wrapper,
    embeddings=embeddings 
)

print(resultados)


Evaluating:  26%|██▋       | 36/136 [04:23<08:24,  5.04s/it]Exception raised in Job[30]: OutputParserException(Invalid json output: Here is the output for the given input:

{
"statements": [
"No information has been found about how the current collection is preserved.",
]
}

Note that since there was only one sentence in the answer, it was broken down into a single statement.
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/OUTPUT_PARSING_FAILURE )
Evaluating: 100%|██████████| 136/136 [16:34<00:00,  7.31s/it]


{'answer_relevancy': 0.4949, 'nv_context_relevance': 0.9853, 'faithfulness': 0.6667, 'nv_response_groundedness': 0.8382}


In [41]:
resultados.to_pandas()

,user_input,retrieved_contexts,response,answer_relevancy,nv_context_relevance,faithfulness,nv_response_groundedness
0,¿Quién fue Joaquín Sorolla?,"[Joaquín Sorolla y Bastida (Valencia, 27 de fe...",Joaquín Sorolla y Bastida fue un pintor españo...,0.685768,1.0,NaN,1.00
1,¿Qué importancia tiene la luz en la pintura de...,[. La amplitud de esta sala bañada por la luz ...,No he encontrado información específica sobre ...,0.500410,1.0,0.666667,0.50
2,¿Cuál es la historia del Museo Sorolla?,[# Datos identificativos del Museo: El Museo S...,La historia del Museo Sorolla comienza con el ...,0.718439,1.0,NaN,1.00
3,¿Dónde se encuentra la casa de Sorolla?,[# El edificio del museo: la casa Sorolla. Des...,La casa de Sorolla se encuentra en el Paseo de...,0.498851,1.0,1.000000,1.00
4,Enumera 5 fechas de la vida de Joaquín Sorolla,[# Fechas importantes de la vida de Joaquín So...,"A continuación, te proporciono 5 fechas import...",0.576831,1.0,NaN,1.00
5,¿Qué influencia tuvo Sorolla en el arte español?,[. Este estaba compuesta en un primer momento ...,No he encontrado información sobre la influenc...,0.613022,1.0,0.500000,0.75
6,¿Quiénes fueron los amigos de Sorolla?,[. En vida de Joaquín Sorolla esta estancia tu...,"Según la información proporcionada, uno de los...",0.000000,1.0,0.500000,1.00
7,¿Cómo se conserva la colección actualmente?,[Las colecciones del museo están compuestas po...,No he encontrado información sobre cómo se con...,0.096994,1.0,NaN,0.75
8,¿Qué relación tenía Sorolla con Valencia?,[Por aquel entonces Valencia lo nombró hijo pr...,Sorolla tenía una relación muy estrecha con Va...,0.642339,1.0,NaN,1.00
9,¿Cuál es el contexto histórico del impresionismo?,"[En agosto de 1900, estando en Valencia, su am...",No he encontrado información sobre el contexto...,0.295095,1.0,NaN,0.50


# Métricas de evaluación de RAGAS con una respuesta esperada


In [18]:
preguntas_respuestas_esperadas = [
    '¿Dónde se encuentra la casa de Sorolla?',
    'Cómo se llama el estilo de pintura de Sorolla?',
    '¿Cómo se llamaban hijos tuvo Joaquín Sorolla?',
    '¿En qué año se inauguró el Museo Sorolla?',
    'Dime el horario de visita',
    'Tarifas de las entradas del museo',
    'Enumera las salas del museo',
    '¿Cuándo nació Joaquín Sorolla?',
    '¿Dónde murió Sorolla?',
    '¿Cuándo falleció Joaquín Sorolla y Bastida?',

]

respuestas_esperadas = [
    'La casa de Sorolla se encuentra en Madrid, en la calle General Martínez Campos.',
    'El estilo de pintura de Sorolla se caracteriza por el uso de la luz natural y los colores vibrantes, a menudo asociado con el impresionismo o luminismo.',
    'Joaquín Sorolla tuvo tres hijos: María, Joaquín y Elena.',
    'El Museo Sorolla se inauguró en 1932.',
    'El horario de visita del Museo Sorolla es de martes a sábado de 9:30 a 20:00 y domingos de 10:00 a 15:00, cerrando los lunes.',
    'Tarifas de las entradas museo: Tarifa general: 3,00 €, Tarifa reducida: 1,50 €, Tarjeta anual de museos estatales: 36,06 €, Tarjeta anual del Museo Sorolla: 25,00 €, Tarjeta ocho museos de Madrid: 16 €, Abono cinco museos: 12 €, Abono cuatro museos: 8 €',
    'Las salas del museo incluyen las Salas I, II, II, el comedor, el salón, el patio andaluz y el jardín.',
    'Joaquín Sorolla nació el 27 de febrero de 1863 en Valencia.',
    'Joaquín Sorolla murió en Cercedilla, Madrid.',
    'Joaquín Sorolla falleció el 10 de agosto de 1923.'
]

In [34]:
df_esperadas = pd.read_csv("interacciones_respuesta.csv")
df_esperadas['reference'] = respuestas_esperadas
df_esperadas.head()

,user_input,retrieved_contexts,response,reference
0,¿Dónde se encuentra la casa de Sorolla?,# El edificio del museo: la casa Sorolla. Desd...,La casa de Sorolla se encuentra en el Paseo de...,"La casa de Sorolla se encuentra en Madrid, en ..."
1,Cómo se llama el estilo artístico de Sorolla?\n\n,.Sorolla no se identificó con los impresionist...,Su estilo artístico ha sido etiquetado como im...,El estilo de pintura de Sorolla se caracteriza...
2,¿Cómo se llamaban hijos tuvo Joaquín Sorolla?,. Este estaba compuesta en un primer momento p...,Los hijos de Joaquín Sorolla se llamaban María...,"Joaquín Sorolla tuvo tres hijos: María, Joaquí..."
3,¿En qué año se inauguró el Museo Sorolla?,". El museo se inauguró en 1932, abriendo al pú...",El Museo Sorolla se inauguró en 1932.,El Museo Sorolla se inauguró en 1932.
4,Dime el horario de visita,# Horarios de visita: De martes a sábado: de 9...,El horario de visita del Museo Sorolla es el s...,El horario de visita del Museo Sorolla es de m...


In [35]:
dataset_reference = []
for i, row in df_esperadas.iterrows():
    entry = {
        "user_input": row['user_input'],
        "retrieved_contexts": row['retrieved_contexts'] if isinstance(row['retrieved_contexts'], list) else [row['retrieved_contexts']],
        "reference": row['reference'],
        "response": row['response']
    }
    dataset_reference.append(entry)

my_eval_dataset_reference = EvaluationDataset.from_list(dataset_reference)

In [39]:
# metricas usadas LLMContextPrecisionWithReference ,LLMContextRecall, FactualCorrectness,AnswerAccuracy
from ragas.metrics import LLMContextPrecisionWithReference, LLMContextRecall, FactualCorrectness, AnswerAccuracy
from ragas.metrics import AnswerRelevancy, ContextRelevance, Faithfulness, ResponseGroundedness


resultados_reference = evaluate(
    dataset=my_eval_dataset_reference,
    metrics=[
        AnswerRelevancy(), 
        ContextRelevance(),
        LLMContextPrecisionWithReference(),
        LLMContextRecall(),
        AnswerAccuracy()
    ],
    llm=llm_wrapper,
    embeddings=embeddings
)

print(resultados_reference)

Evaluating: 100%|██████████| 50/50 [04:38<00:00,  5.56s/it]


{'answer_relevancy': 0.5307, 'nv_context_relevance': 1.0000, 'llm_context_precision_with_reference': 0.9000, 'context_recall': 0.9500, 'nv_accuracy': 0.9500}


In [40]:
resultados_reference.to_pandas()

,user_input,retrieved_contexts,response,reference,answer_relevancy,nv_context_relevance,llm_context_precision_with_reference,context_recall,nv_accuracy
0,¿Dónde se encuentra la casa de Sorolla?,[# El edificio del museo: la casa Sorolla. Des...,La casa de Sorolla se encuentra en el Paseo de...,"La casa de Sorolla se encuentra en Madrid, en ...",0.498851,1.0,1.0,1.0,1.0
1,Cómo se llama el estilo artístico de Sorolla?\n\n,[.Sorolla no se identificó con los impresionis...,Su estilo artístico ha sido etiquetado como im...,El estilo de pintura de Sorolla se caracteriza...,0.454785,1.0,1.0,1.0,1.0
2,¿Cómo se llamaban hijos tuvo Joaquín Sorolla?,[. Este estaba compuesta en un primer momento ...,Los hijos de Joaquín Sorolla se llamaban María...,"Joaquín Sorolla tuvo tres hijos: María, Joaquí...",0.545915,1.0,1.0,1.0,1.0
3,¿En qué año se inauguró el Museo Sorolla?,"[. El museo se inauguró en 1932, abriendo al p...",El Museo Sorolla se inauguró en 1932.,El Museo Sorolla se inauguró en 1932.,0.780472,1.0,1.0,1.0,1.0
4,Dime el horario de visita,[# Horarios de visita: De martes a sábado: de ...,El horario de visita del Museo Sorolla es el s...,El horario de visita del Museo Sorolla es de m...,0.413788,1.0,1.0,1.0,1.0
5,Tarifas de las entradas del museo,[Servicios del Museo\n\nEl 27 de abril de 1973...,Las tarifas de las entradas del Museo Sorolla ...,Tarifas de las entradas museo: Tarifa general:...,0.449477,1.0,1.0,1.0,1.0
6,Enumera las salas del museo,[Servicios del Museo\n\nPara la habilitación d...,Las salas del museo mencionadas son:\n\n1. Sal...,"Las salas del museo incluyen las Salas I, II, ...",0.452490,1.0,0.0,0.5,0.5
7,¿Cuándo nació Joaquín Sorolla?,[# Fechas importantes de la vida de Joaquín So...,Joaquín Sorolla nació el 27 de febrero de 1863...,Joaquín Sorolla nació el 27 de febrero de 1863...,0.607027,1.0,1.0,1.0,1.0
8,¿Dónde murió Joaquín Sorolla?,"[Joaquín Sorolla y Bastida (Valencia, 27 de fe...","Joaquín Sorolla murió en Cercedilla, el 10 de ...","Joaquín Sorolla murió en Cercedilla, Madrid.",0.630401,1.0,1.0,1.0,1.0
9,¿Cuándo falleció Joaquiín Sorolla y Bastida?,"[Joaquín Sorolla y Bastida (Valencia, 27 de fe...",Joaquín Sorolla y Bastida falleció el 10 de ag...,Joaquín Sorolla falleció el 10 de agosto de 1923.,0.473911,1.0,1.0,1.0,1.0


In [63]:
resultados_reference.to_pandas().mean(numeric_only=True).to_frame().T


,answer_relevancy,nv_context_relevance,llm_context_precision_with_reference,context_recall,nv_accuracy
0,0.530712,1.0,0.9,0.95,0.95
